# Contents
- [Introduction](#Introduction)
- [Combining Multiple Blockers](#Combining-Multiple-Blockers)

# Introduction

This IPython notebook illustrates how to use multiple blockers and combine the results.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
# Import py_entitymatching package
import py_entitymatching as em
import os
import pandas as pd

/Users/pradap/miniconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Then, read the (sample) input tables for blocking purposes.

In [2]:
# Get the datasets directory
datasets_dir = em.get_install_path() + os.sep + 'datasets'

# Get the paths of the input tables
path_A = datasets_dir + os.sep + 'person_table_A.csv'
path_B = datasets_dir + os.sep + 'person_table_B.csv'

In [3]:
# Read the CSV files and set 'ID' as the key attribute
A = em.read_csv_metadata(path_A, key='ID')
B = em.read_csv_metadata(path_B, key='ID')

# Combining Multiple Blockers

In [4]:
#Blocking plan :

#A, B --overlap blocking--> candset --attr-equiv-block--> candset  |
#                                                                  | 
#A, B ------------rule-based-blocking--------------------> candset |----union--->candset
#                                                                  |
#A, B -----------black-box-blocking----------------------> candset |


In [5]:
# Overlap blocking over input tables
ob = em.OverlapBlocker()
# block using name
C = ob.block_tables(A, B, 'name', 'name', word_level=True, overlap_size=1, 
                    l_output_attrs=['name', 'birth_year'], 
                    r_output_attrs=['name', 'birth_year'])
C

0%  100%
[######] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_ID,rtable_ID,ltable_name,ltable_birth_year,rtable_name,rtable_birth_year
0,0,a3,b2,William Bridge,1986,Bill Bridge,1986
1,1,a2,b3,Michael Franklin,1988,Mike Franklin,1988
2,2,a5,b5,Alphonse Kemper,1984,Alfons Kemper,1984
3,3,a2,b6,Michael Franklin,1988,Michael Brodie,1987


In [6]:
# Overlap blocking over input tables
ob = em.OverlapBlocker()
# block using name
C = ob.block_tables(A, B, 'name', 'name', word_level=True, overlap_size=1, 
                    l_output_attrs=['name', 'birth_year'], 
                    r_output_attrs=['name', 'birth_year'])
C

0%  100%
[######] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_ID,rtable_ID,ltable_name,ltable_birth_year,rtable_name,rtable_birth_year
0,0,a3,b2,William Bridge,1986,Bill Bridge,1986
1,1,a2,b3,Michael Franklin,1988,Mike Franklin,1988
2,2,a5,b5,Alphonse Kemper,1984,Alfons Kemper,1984
3,3,a2,b6,Michael Franklin,1988,Michael Brodie,1987


In [7]:
# Attribute equivalence blocking: block C using birth_year
ab = em.AttrEquivalenceBlocker()
D = ab.block_candset(C, 'birth_year', 'birth_year')

0%  100%
[####] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [8]:
D

,_id,ltable_ID,rtable_ID,ltable_name,ltable_birth_year,rtable_name,rtable_birth_year
0,0,a3,b2,William Bridge,1986,Bill Bridge,1986
1,1,a2,b3,Michael Franklin,1988,Mike Franklin,1988
2,2,a5,b5,Alphonse Kemper,1984,Alfons Kemper,1984


In [10]:
# Rule-based blocking over input tables
# first get features that can be used
feature_table = em.get_features_for_blocking(A, B)

In [11]:
# Create rule-based blocker
rb = em.RuleBasedBlocker()
# Add rule : block tuples if name_name_lev(ltuple, rtuple) < 0.4
rb.add_rule(['name_name_lev_sim(ltuple, rtuple) < 0.4'], feature_table)

'_rule_0'

In [19]:
E = rb.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'])

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [12]:
# Apply black box blocker
# Create black box blocker
bb = em.BlackBoxBlocker()

In [13]:
# define a black box function. 
# The blocker function should drop tuple pairs whose last name do not match
# The function has to do the following steps
#  1) Get name attributes from each of the tuples
#  2) Split name attribute to get last name
#  3) if last names donot match return True

In [14]:
def my_function(x, y):
    # x, y will be of type pandas series
    
    # get name attribute
    x_name = x['name']
    y_name = y['name']
    # get last names
    x_name = x_name.split(' ')[1]
    y_name = y_name.split(' ')[1]
    # check if last names match
    if x_name != y_name:
        return True
    else:
        return False

In [15]:
bb.set_black_box_function(my_function)

In [16]:
F = bb.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'])

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [17]:
F

,_id,ltable_ID,rtable_ID,ltable_name,rtable_name
0,0,a2,b3,Michael Franklin,Mike Franklin
1,1,a3,b2,William Bridge,Bill Bridge
2,2,a5,b5,Alphonse Kemper,Alfons Kemper


In [20]:
# Combine all the blocker outputs
G = em.combine_blocker_outputs_via_union([D, E, F])

In [21]:
G

,_id,ltable_ID,rtable_ID,ltable_name,ltable_birth_year,rtable_name,rtable_birth_year
0,0,a2,b3,Michael Franklin,1988,Mike Franklin,1988
1,1,a2,b6,Michael Franklin,1988,Michael Brodie,1987
2,2,a3,b2,William Bridge,1986,Bill Bridge,1986
3,3,a3,b6,William Bridge,1986,Michael Brodie,1987
4,4,a4,b2,Binto George,1987,Bill Bridge,1986
5,5,a5,b5,Alphonse Kemper,1984,Alfons Kemper,1984


In [22]:
em.show_properties(G)

id: 4643013464
fk_ltable: ltable_ID
fk_rtable: rtable_ID
rtable(obj.id): 4642423864
ltable(obj.id): 4642423080
key: _id
